Chapter 7 Deques and Linked Lists<br><br>

A deque is a doubly-ended queue. It acts like both a stack and a queue, storing an ordered collection of items with the ability to add or remove from both the beginning and the end.<br>

7.1 The Deque ADT
- addfirst(item) - add item to the front of the deque.
- addlast(item) - add item to the end of the deque.
- removefirst(item) - remove and return the first item in the deque.
- removelast(item) - remove and return the last item in the deque.
- len - return the number of items in the deque

In [1]:
#Deque implementation using list
class ListDeque:
    def __init__(self):
        self._L = []
    
    def addfirst(self, item):
        self._L.insert(0, item)

    def addlast(self, item):
        self._L.append(item)

    def removefirst(self):
        return self._L.pop(0)
    
    def removelast(self):
        return self._L.pop()
    
    def __len__(self):
        return len(self._L)

The code is simple enough, but a couple of operations will start to get slow as the deque gets large. Inserting and popping at index 0 takes O(n) time. This follows from the way lists are stored sequentially in memory. There is no way to change the beginning of the list without shifting all the other elements to make room or fill gaps. To make this work, we are going to have to give up on the idea of having the items laid out sequentially in memory. Instead, we’ll store some more information with each item that we can use to extract the order.

7.2 Linked Lists<br><br>

Linked lists are a simple data structure for storing a sequential collection. Unlike a standard Python list, it will allow us to insert at the beginning quickly. The idea is to store the items in individual objects called nodes. A special node is the head of the list. Each node stores a reference to the next node in the list.

In [2]:
class ListNode:
    def __init__(self, data, link=None):
        self.data = data
        self.link = link

To start the LinkedList, we will store the head of the list. We will provide two methods, addfirst and removefirst both of which modify the beginning of the list. These will behave roughly like the push and pop operations of the stack. This first implementation will hide the nodes from the user. That is, from a users perspective, they can create a linked list, and they can add and remove nodes, but they don’t have to touch (or even know about) the nodes. This is abstraction (hiding details)!

In [3]:
class LinkedList:
    def __init__(self):
        self._head = None

    def addfirst(self, item):
        self._head = ListNode(item, self._head)

    def removefirst(self):
        item = self._head.data
        self._head = self._head.link
        return item

In [4]:
LL = LinkedList()
LL.addfirst(1)
LL.addfirst(2)
LL.removefirst()

2

7.3 Implementing a Queue with a LinkedList<br><br>

Our best list implementation of a Queue required linear time in the worst case for dequeue operations We could hope to do better with a linked list. However, right now, we have no way to add to or remove from the end of the linked list. Here is an inefficient, though simple and correct way to do it.

In [5]:
class LinkedList:
    def __init__(self):
        self._head = None

    def addfirst(self, item):
        self._head = ListNode(item, self._head)

    def addlast(self, item):
        if self._head is None:
            self.addfirst(item)

        else:
            currentnode = self._head
            while currentnode.link is not None:
                currentnode = currentnode.link
            currentnode.link = ListNode(item)

    def removefirst(self):
        item = self._head.data
        self._head = self._head.link
        return item
    
    def removelast(self):
        if self._head.link is None:
            self.removefirst()

        else:
            currentnode = self._head
            while currentnode.link.link is not None:
                currentnode = currentnode.link
            item = currentnode.link.data
            currentnode.link = None
            return item

In [6]:
LL2 = LinkedList()
LL2.addfirst(3)
LL2.addfirst(2)
LL2.addlast(1)
LL2.removefirst()

2

The new addlast method starts at the head of the linked list and traverses to the end by following the links. It uses the convention that the link of the last node is None.
Similarly, removelast traverses the list until it reaches the second to last element. This traversal approach is not very efficient. For a list of length n, we would need O(n) time to find the end.
A different approach might be to store the last node (or tail) of the list so we don’t have to search for it when we need it.
We will be able to use this to get some improvement for addlast, because we will be able to jump right to the end without traversing.

In [7]:
class LinkedList:
    def __init__(self):
        self._head = None
        self._tail = None

    def addfirst(self, item):
        self._head = ListNode(item, self._head)
        if self._tail is None: self._tail = self._head

    def addlast(self, item):
        if self._head is None:
            self.addfirst(item)
        
        else:
            self._tail.link = ListNode(item)
            self._tail = self._tail.link
    
    def removefirst(self):
        item = self._head.data
        self._head = self._head.link
        if self._head is None: self._tail = None
        return item
    
    def removelast(self):
        if self._head is self._tail:
            self.removefirst()

        else:
            currentnode = self._head
            while currentnode.link is not self._tail:
                currentnode = currentnode.link
            item = self._tail.data
            self._tail = currentnode
            self._tail.link = None
            return item

Now we can implement the Queue ADT with a linked list.

In [8]:
from ds2.deque import LinkedList

class LinkedQueue:
    def __init__(self):
        self._L = LinkedList()

    def enqueue(self, item):
        self._L.addlast(item)

    def dequeue(self):
        return self._L.removefirst()

    def peek(self):
        item = self._L.removefirst()
        self._L.addfirst(item)
        return item

    def __len__(self):
        return len(self._L)

    def isempty(self):
        return len(self) == 0

7.4 Storing the length<br><br>

we will want delegate the length computation to the LinkedList.

In [9]:
class LinkedList:
    def __init__(self):
        self._head = None
        self._tail = None
        self._lenght = 0

    def addfirst(self, item):
        self._head = ListNode(item, self._head)
        if self._tail is None: self._tail = self._head
        self._lenght += 1

    def addlast(self, item):
        if self._head is None:
            self.addfirst(item)

        else:
            self._tail.link = ListNode(item)
            self._tail = self._tail.link
            self._lenght += 1

    def removefirst(self):
        item = self._head.data
        self._head = self._head.link
        if self._head is None: self._tail = None
        self._lenght -= 1
        return item
    
    def removelast(self):
        if self._head is self._tail:
            self.removefirst()
        
        else:
            currentnode = self._head
            while currentnode is not self._tail:
                currentnode = currentnode.link
            item = self._tail.data
            self._tail = currentnode
            self._tail.link = None
            self._lenght -= 1
            return item
        
    def __len__(self):
        return self._lenght
    
    def isempty(self):
        return self._lenght == 0

We still have to iterate through the whole list in order to remove from the end. It seems very hard to avoid this. As a result, our removelast method still takes linear time.

7.5 Testing Against the ADT

In [6]:
import unittest
from ds2.queue import ListQueue

class TestListQueue(unittest.TestCase):
    def testinit(self):
        q = ListQueue()

    def testaddandremoveoneitem(self):
        q = ListQueue()
        q.enqueue(3)
        self.assertEqual(q.dequeue(), 3)
    
    def testalternatingaddremove(self):
        q = ListQueue()
        for i in range(1000):
            q.enqueue(i)
            self.assertEqual(q.dequeue(), i)
    
    def testmanyoperations(self):
        q = ListQueue()
        for i in range(1000):
            q.enqueue(2 * i + 3)
        for i in range(1000):
            self.assertEqual(q.dequeue(), 2 * i + 3)

    def testlength(self):
        q = ListQueue()
        self.assertEqual(len(q), 0)
        for i in range(10):
            q.enqueue(i)
        self.assertEqual(len(q), 10)
        for i in range(10):
            q.enqueue(i)
        self.assertEqual(len(q), 20)
        for i in range(15):
            q.dequeue()
        self.assertEqual(len(q), 5)

if __name__ == "__main__":
    unittest.main()

.....<br>
\----------------------------------------------------------------------<br>
Ran 5 tests in 0.003s<br><br>
OK

We wanted to copy a bunch of methods to be included in two different classes (TestListQueue and TestLinkedQueue). Instead we want them to share the methods. So, we refactor the code, by factoring out a superclass.

In [2]:
#testqueue.py
class QueueTests:
    def __init__(self):
        q = self.Queue()

    def testaddandremoveoneitem(self):
        q = self.Queue()
        q.enqueue(3)
        self.assertEqual(q.dequeue(), 3)

    def testalternatingaddremove(self):
        q = self.Queue()
        for i in range(1000):
            q.enqueue(i)
            self.assertEqual(q.dequeue(), i)
    
    def testmanyoperations(self):
        q = self.Queue()
        for i in range(1000):
            q.enqueue(2 * i + 3)
        for i in range(1000):
            self.assertEqual(q.dequeue(), 2 * i + 3)

    def testlength(self):
        q = self.Queue()
        self.assertEqual(len(q), 0)
        for i in range(10):
            q.enqueue(i)
        self.assertEqual(len(q), 10)
        for i in range(10):
            q.enqueue(i)
        self.assertEqual(len(q), 20)
        for i in range(15):
            q.dequeue()
        self.assertEqual(len(q), 5)

Instead of creating a new ListQueue object in each test, we create a self.Queue object. <br>This may seem strange for two reasons: first, we don’t have this class; and second, it is attached to self which is the TestQueue object. When we implement the specific tests for each class, we will assign the variable Queue to be the class corresponding to the particular implementation we want to test.

In [6]:
# testlistqueue.py
import unittest
from ds2.test.testqueue import QueueTests
from ds2.queue import ListQueue

class TestListQueue(unittest.TestCase, QueueTests):
    Queue = ListQueue

if __name__ == "__main__":
    unittest.main()

.....<br>
\----------------------------------------------------------------------<br>
Ran 5 tests in 0.003s<br><br>
OK

In [7]:
# testlinkedqueue.py
import unittest
from ds2.test.testqueue import QueueTests
from ds2.queue import LinkedQueue

class TestLinkedQueue(unittest.TestCase, QueueTests):
    Queue = LinkedQueue

if __name__ == "__main__":
    unittest.main()

.....<br>
\----------------------------------------------------------------------<br>
Ran 5 tests in 0.003s<br><br>
OK

The classes, TestListQueue and TestLinkedQueue, extend both unittest.TestCase and TestQueue. This is called multiple inheritance.

In [2]:
# testbothqueue.py
import unittest
from ds2.test.testqueue import QueueTests
from ds2.queue import ListQueue, LinkedQueue

def _test(queue_class):
    class QueueTestCase(unittest.TestCase, QueueTests):
        Queue = queue_class
    return QueueTestCase

TestLinkedQueue = _test(LinkedQueue)
TestListQueue = _test(ListQueue)

if __name__ == "__main__":
    unittest.main()

.....<br>
\----------------------------------------------------------------------<br>
Ran 10 tests in 0.008s<br><br>
OK

Ten tests will execute, five for each implementation.

7.6 The Main Lessons:
- Use the public interface as described in an ADT to test your class.
- You can use inheritance to share functionality between classes.
- Inheritance means ’is a’.

7.7 Design Patterns: The Wrapper Pattern

We create a new class that has an instance of another class (alist or LinkedList in our example) and then provide methods that operate on that object. From outside the class, we don’t have to know anything about the wrapped class. Sometimes, this separation is called a layer of abstraction. The user of our class does not have to know anything about our implementation in order to use the class. <br>
Here are some of the takeaway:
- Use design patterns where appropriate to organize your code and improve readability.
- The Wrapper Pattern gives a way to provide an alternative interface to (a subset of) the methods in another class.
- Composition means ’has a’.